In [23]:
import numpy as np
import cv2
from PIL import Image

In [24]:
def VC(p_i, k_i, n):
    """
    p_i: число для шифрования
    k_i: ключ (задает смещение в алфавите)
    n: число бит для шифрования пикселя картинки
    """
    c_i = (p_i + k_i) % (2**n)
    return c_i


def VD(c_i, k_i, n):
    """
    c_i: зашифрованное число
    k_i: ключ (задает смещение в алфавите)
    n: число бит для шифрования пикселя картинки
    """
    p_i = (c_i - k_i) % (2**n)
    return p_i


def Arnold(matrix):
    """
    matrix[m x m]: матрица, к которой применяется преобразование Арнольда
    """
    m = len(matrix)
    new_matrix = np.zeros((m, m), dtype=int)
    for x in range(m):
        for y in range(m):
            new_matrix[(x + y) % m][(x + 2*y) % m] = matrix[x][y]
    return new_matrix


def key_expansion(k1, k2, k3, k4):
    """
    k1, k2, k3, k4: вектора размерности [16]
    """
    V1, V3 = k1[0] ^ k1[1], k3[0] ^ k3[1]
    V2, V4 = VC(k2[0], k2[1], 8), VC(k4[0], k4[1], 8)

    for i in range(2, 16):
        V1 = V1 ^ k1[i]  # XOR
        V3 = V3 ^ k3[i]  # XOR

        V2 = VC(V2, k2[i], 8)
        V4 = VC(V4, k4[i], 8)
    
    o1 = np.zeros((16, 16), dtype=int)
    o2 = np.zeros((16, 16), dtype=int)
    o = np.zeros((16, 16), dtype=int)
    key_B = np.zeros((16, 16), dtype=int)

    for i in range(16):
        for j in range(16):
            o1[i][j] = VC(k1[i], VC(V2, V1, 8), 8) ^ VC(k2[j], V2 ^ V1, 8)
            o2[i][j] = VC(k3[i], VC(V4, V3, 8), 8) ^ VC(k4[j], V4 ^ V3, 8)
            o[i][j] = o1[i][j] ^ o2[i][j]
    
    arnold_iterations = (o1[1][1] + o1[1][1]) % (2**8)
    
    for i in range(arnold_iterations):
        key_B = Arnold(o)
        
    return key_B


def ciphering_block(B, key_B, n):
    """
    B[16 x 16]: блок для шифрования
    key_B[16 x 16]: ключ шифрования блока
    n: число бит для шифрования пикселя картинки
    """
    B_C = np.zeros((16, 16), dtype=np.uint8)

    for i in range(16):
        for j in range(16):
            B_C[i][j] = VC(B[i][j], key_B[i][j], n)

    return B_C


def deciphering_block(B_C, key_B, n):
    """
    B_C[16 x 16]: зашифрованный блок
    key_B[16 x 16]: ключ расшифрования блока
    n: число бит для шифрования пикселя картинки
    """
    B_D = np.zeros((16, 16), dtype=np.uint8)

    for i in range(16):
        for j in range(16):
            B_D[i][j] = VD(B_C[i][j], key_B[i][j], n)

    return B_D


def s(k1, k2, k3, k4):
    k = np.vstack([k1, k2, k3, k4]).T
    return np.dot(k,k.T)

def key_update(key_B, B_C, B, k1, k2, k3, k4, n):
    """
    B[16 x 16]: предыдущий блок B
    B_C[16 x 16]: зашифрованный блок В
    key_B[16 x 16]: ключ для шифрования предыдущего блока B
    k1, k2, k3, k4: вектора размерности [16]
    n: число бит для шифрования пикселя картинки
    """
    key_B = np.array(key_B)
    B_C = np.array(B_C)
    B_D = np.array(B)
    k1 = np.array(k1)
    k2 = np.array(k2)
    k3 = np.array(k3)
    k4 = np.array(k4)
    return ((key_B + np.outer(B_C.sum(0),B_D.sum(1)) + s(k1, k2, k3, k4)) % (2**n)).tolist()

In [25]:
def blockshaped(arr, nrows, ncols):
    """
    Return an array of shape (n, nrows, ncols) where
    n * nrows * ncols = arr.size

    If arr is a 2D array, the returned array should look like n subblocks with
    each subblock preserving the "physical" layout of arr.
    """
    h, w = arr.shape
    assert h % nrows == 0, "{} rows is not evenly divisble by {}".format(h, nrows)
    assert w % ncols == 0, "{} cols is not evenly divisble by {}".format(w, ncols)
    return (arr.reshape(h//nrows, nrows, -1, ncols)
               .swapaxes(1,2)
               .reshape(-1, nrows, ncols))

In [26]:
k1 = [1,32,3,234,15,156,7,8,9,40,1,2,3,43,5,64]
k2 = [1,2,35,54,55,26,73,8,9,50,21,2,3,45,5,65]
k3 = [1,27,3,24,25,6,7,8,9,0,13,2,23,4,5,64]
k4 = [143,25,33,4,5,6,73,8,39,0,1,2,3,4,5,6]

img_path = 'img.jpg'
img = cv2.imread(img_path, 0)

imgb = blockshaped(img, 16, 16)
Nb = len(imgb)
imgb_c = [0] * Nb
imgb_d = [0] * Nb

key_B = key_expansion(k1, k2, k3, k4)

for i in range(Nb):
    B = imgb[i]
    B_C = ciphering_block(B, key_B, 8)
    imgb_c[i] = B_C
    key_update(key_B, B_C, B, k1, k2, k3, k4, 8)

In [27]:
print(key_B) # ключ шифрования первого блока (дальше обновляем в key_update)

[[ 40  37   8 157 100 150  26 102 232 108 146  40 136  87 134 244]
 [121 173 167 130 118 155 132   0 152   2 110 144 188  22 109 190]
 [132  65  32  34   0 252 112 123  18 130 252 132 146 190 130 243]
 [103  26 123  24 175 133 126 250 144 237 144 230 122 120 188 128]
 [130 101 142 229  34 151   8 251 120  26   6 111 244  96 134  86]
 [168 104 103 140 113 188 173  48 118 253 152 140 132  11 114 156]
 [142 178 150 141 142 115  40  51  10  78 112  29  14  14 224 141]
 [102 113 160 140 115 100 113  42 167 148 116  72 144 139 140 106]
 [232 236 154  95 158 105 154 155  40 165   0 234 114 168   6   9]
 [132 109 110  16 180  97 123 128 101 194 167   2 126 236 146  62]
 [  4 188 224 235 146  62 138 132 146 127  60  77   0 124 120  12]
 [152 154 134 216 102  23 188   0 111 109 109  38 179 234 126 122]
 [120 154  14  24 226  94 154  57 130 229 134 146  52 169  20 148]
 [106 146 152  12 140 124 100 162 180   7 103  12 121 203 187  14]
 [ 28 112 108 114  14 142 232 250 152 140 138 226 142 243  32 

In [28]:
print(imgb_c[2]) # Зашифрованный блок 16 х 16

[[ 72  72  39 191 134 182  61 135   9 141 179  73 169 120 167  22]
 [153 207 199 163 151 188 166  33 185  35 143 177 221  55 143 224]
 [165  97  66  67  33  30 145 157  51 163  29 165 179 224 164  21]
 [136  57 158  56 207 168 158  28 177  14 177   7 156 154 222 162]
 [163 132 177   5  66 186  40  29 153  59  39 144  22 130 168 120]
 [201 136 137 173 146 222 206  82 151  30 185 174 166  45 148 190]
 [174 212 182 174 175 148  74  84  43 111 146  63  48  48   2 175]
 [134 148 191 174 149 132 148  75 200 182 150 106 178 173 174 140]
 [ 11  13 186 128 190 136 187 191  75 199  33  10 147 202  42  46]
 [155 140 146  50 212 131 157 160 134 227 201  36 161  15 181  97]
 [ 21 210 253  14 183  97 170 164 177 159  95 113  37 160 154  45]
 [172 170 152 244 138  59 221  31 143 142 143  73 215  13 161 156]
 [140 172  29  40 250 127 189  88 164   7 167 179  85 203  56 185]
 [126 165 168  26 156 146 129 196 215  41 136  44 153 237 223  52]
 [ 47 129 125 133  31 157 252  23 186 174 171   4 176  22  67 

In [29]:
print(imgb[47]) # Исходный блок 16 х 16

[[23 27 32 37 41 43 43 43 42 42 42 42 42 42 42 42]
 [23 26 32 37 41 43 43 42 42 42 42 42 42 42 42 42]
 [23 26 32 37 41 43 43 42 41 41 42 42 42 42 42 42]
 [23 26 32 37 41 43 43 42 41 41 41 41 42 42 42 42]
 [23 26 32 37 41 43 43 42 41 41 41 41 41 41 42 42]
 [23 26 31 37 41 42 42 42 41 41 41 41 41 41 41 41]
 [22 26 31 37 41 42 42 42 40 40 41 41 41 41 41 41]
 [22 26 31 37 41 42 42 42 40 40 40 41 41 41 41 41]
 [23 26 31 36 39 41 40 40 40 40 40 40 40 40 40 40]
 [23 26 31 36 39 40 40 39 40 40 40 40 40 40 40 40]
 [22 25 30 35 38 40 39 39 39 39 39 39 39 39 39 39]
 [21 24 29 34 38 39 38 38 38 38 38 38 38 38 38 38]
 [20 24 29 34 37 38 38 37 37 37 37 37 37 37 37 37]
 [20 23 28 33 36 37 37 36 36 36 36 36 36 36 36 36]
 [19 22 27 32 36 37 36 36 36 36 36 36 36 36 36 36]
 [19 22 27 32 35 36 36 35 35 35 35 35 35 35 35 35]]


In [30]:
# дешифрование зашифрованной картинки
for i in range(Nb):
    B_C = imgb_c[i]
    B_D = deciphering_block(B_C, key_B, 8)
    imgb_d[i] = B_D
    key_update(key_B, B_C, B_D, k1, k2, k3, k4, 8)

In [31]:
print(imgb_d[47]) # Дешифрованный блок 16 х 16 (получен из imgb_c в ходе дешифровки)

[[23 27 32 37 41 43 43 43 42 42 42 42 42 42 42 42]
 [23 26 32 37 41 43 43 42 42 42 42 42 42 42 42 42]
 [23 26 32 37 41 43 43 42 41 41 42 42 42 42 42 42]
 [23 26 32 37 41 43 43 42 41 41 41 41 42 42 42 42]
 [23 26 32 37 41 43 43 42 41 41 41 41 41 41 42 42]
 [23 26 31 37 41 42 42 42 41 41 41 41 41 41 41 41]
 [22 26 31 37 41 42 42 42 40 40 41 41 41 41 41 41]
 [22 26 31 37 41 42 42 42 40 40 40 41 41 41 41 41]
 [23 26 31 36 39 41 40 40 40 40 40 40 40 40 40 40]
 [23 26 31 36 39 40 40 39 40 40 40 40 40 40 40 40]
 [22 25 30 35 38 40 39 39 39 39 39 39 39 39 39 39]
 [21 24 29 34 38 39 38 38 38 38 38 38 38 38 38 38]
 [20 24 29 34 37 38 38 37 37 37 37 37 37 37 37 37]
 [20 23 28 33 36 37 37 36 36 36 36 36 36 36 36 36]
 [19 22 27 32 36 37 36 36 36 36 36 36 36 36 36 36]
 [19 22 27 32 35 36 36 35 35 35 35 35 35 35 35 35]]


In [32]:
k = int(len(img[0]) / 16)
arr = []

for i in range(0, Nb, k): # cобираем блоки [16 х 16] по k штук в массивы
    arr.append(imgb_c[i:i+k])

arr1 = [0] * len(arr)

for i in range(0, len(arr)): # cобираем блоки [16 х 16] в строки
    for j in range(len(arr[0])):
        if j == 0:
            arr1[i] = arr[i][j]
        else:
            arr1[i] = np.concatenate((arr1[i], arr[i][j]), axis=1)

for i in range(len(arr1)): # cобираем строки [16 х (16*k)] в картинку
    if i == 0:
        arr2 = arr1[i]
    else:
        arr2 = np.concatenate((arr2, arr1[i]), axis=0)

        
im_1 = Image.fromarray(arr2, 'L')
im_1.save("decrypted.jpg")